In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
from twisted.internet import reactor
from txdbus import error, client


class Notifier:
    # TODO: Can this be generalized to load arbitrary resources?
    @classmethod
    async def create(xls, conn=None):
        if not conn:
            conn = await client.connect(reactor, 'session')
        
        notifier = await conn.getRemoteObject(
            'org.freedesktop.Notifications',
            '/org/freedesktop/Notifications'
        )
        
        return Notifier(conn=conn, notifier=notifier)
      
    def __init__(self, conn, notifier):
        self.conn = conn
        self.notifier = notifier
        
    def notify(self, *, replaces=0, appname, summary, message, timeout):
        return self.notifier.callRemote(
            'Notify',
            appname,
            replaces,
            '',
            summary,
            message,
            [], dict(),
            timeout
        )

notifier = await Notifier.create()

In [3]:
id = await notifier.notify(
    appname='pyxsession',
    summary='an important message',
    message='my dog',
    timeout=5000
)

In [4]:
# default the busAddress parameter to "session"
# this address can be "session", "system", or
# something more specific - either a unix, tcp
# or nonce-tcp connection type, apparently
#
# for us, session is perf 👍
conn = await client.connect(reactor)

print('Connected to the session bus!')

Connected to the session bus!


In [5]:
from twisted.internet import reactor
from txdbus import client

# txdbus can reflect off remote objects without a specified schema
remote_obj = await conn.getRemoteObject('org.example', '/MyObjPath')

result = await remote_obj.callRemote('Ping')

print(result)

IntrospectionFailed: Introspection Failed: org.freedesktop.DBus.Error.ServiceUnknown: The name org.example was not provided by any .service files

In [6]:
# Calling remote on the conn directly - no reflection but
# more flexible
result = await conn.callRemote('/AnyValidObjPath', 'Ping',
                               interface='org.freedesktop.DBus.Peer',
                               destination='org.freedesktop.Notifications')

print(result)

None


In [7]:
from txdbus.interface import DBusInterface, Method

peer_iface = DBusInterface(
    'org.freedesktop.DBus.Peer',
    # This method takes no arguments
    Method('Ping')
    # can also do things like Method('full', arguments='s', returns='i') for a function str -> int
    # also a Signal type that I think is for pubsub but takes the same arguments
)

remote_obj = await conn.getRemoteObject('org.freedesktop.Notifications', '/MyObjPath', peer_iface)

await remote_obj.callRemote('Ping')

In [8]:
from twisted.internet import reactor, defer
from txdbus import client, objects
from txdbus.interface import DBusInterface, Method

class MyObj(objects.DBusObject):
    iface = DBusInterface(
        'org.example.MyIFace',
        Method('exampleMethod', arguments='s', returns='s')
    )
    
    dbusInterfaces = [iface]
    
    def __init__(self, objectPath):
        super().__init__(objectPath)
        
    def dbus_exampleMethod(self, arg):
        print(f'received: {arg}')
        return f'You sent: {arg}'

conn.exportObject(MyObj('/MyObjPath'))

await conn.requestBusName('org.example')

1

In [9]:
remote_obj = await conn.getRemoteObject('org.example', '/MyObjPath')

reply = await remote_obj.callRemote('exampleMethod', 'ping!')

reply

received: ping!


'You sent: ping!'

In [10]:
from txdbus.interface import Property
from txdbus.objects import DBusProperty

class MyOtherObj(objects.DBusObject):
    iface = DBusInterface(
        'org.example.MyOtherIFace',
        Property('foo', 's', writeable=True)
    )
    
    dbusInterfaces = [iface]
    
    foo = DBusProperty('foo')
    
    def __init__(self, objectPath):
        super().__init__(objectPath)
        
        self.foo = 'bar'

conn.exportObject(MyOtherObj('/MyOtherObjPath'))

await conn.requestBusName('org.example')

4

In [11]:
remote_obj = await conn.getRemoteObject('org.example', '/MyOtherObjPath')

foo = await remote_obj.callRemote('Get', '', 'foo')

print(foo)

await remote_obj.callRemote('Set', '', 'foo', 'baz')

await remote_obj.callRemote('Get', '', 'foo')

bar


'baz'

In [21]:
from twisted.internet.defer import ensureDeferred
from functools import wraps

def returns_deferred(f):
    @wraps(f)
    def wrapped(*args, **kwargs):
        return ensureDeferred(f(*args, **kwargs))
      
    return wrapped
  

# This doesn't work because my lil' example object doesn't
# support it I guess?
class IdentityExample(objects.DBusObject):
    iface = DBusInterface(
        'org.example.IdentityExampleIFace',
        Method('identityExample', returns='s')
    )
    
    dbusInterfaces = [iface]
    
    @returns_deferred
    async def dbus_identityExample(self, dbusCaller=None):
        if dbusCaller:
            user = await self.getConnection().getConnectionUnixUser(dbusCaller)
            
        else:
            user = None
            
        return f'Your user: {user}'
      
conn.exportObject(IdentityExample('/IdentityExample'))

await conn.requestBusName('org.example')

4

In [22]:
identity_remote = await conn.getRemoteObject('org.example', '/IdentityExample')

await identity_remote.callRemote('identityExample')

'Your user: None'